<a href="https://colab.research.google.com/github/navneetkrc/Colab_fastai/blob/master/PyTorch_basics_Linear_Regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch basics - Linear Regression from scratch

<iframe width="560" height="315" src="https://www.youtube.com/embed/ECHX1s0Kk-o?controls=0" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

Tutorial inspired from [FastAI development notebooks](https://github.com/fastai/fastai_v1/tree/master/dev_nb)

## Machine Learning

<img src="https://i.imgur.com/oJEQe7k.png" width="500">


## Tensors & Gradients

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# Import Numpy & PyTorch
import numpy as np
import torch

A tensor is a number, vector, matrix or any n-dimensional array.

In [0]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [57]:
# Print tensors
print(x)
print(w)
print(b)

tensor(3.)
tensor(4., requires_grad=True)
tensor(5., requires_grad=True)


We can combine tensors with the usual arithmetic operations.

In [58]:
# Arithmetic operations
y = w * x + b
print(y)

tensor(17., grad_fn=<ThAddBackward>)


What makes PyTorch special, is that we can automatically compute the derivative of `y` w.r.t. the tensors that have `requires_grad` set to `True` i.e. `w` and `b`.

In [0]:
# Compute gradients
y.backward()

In [60]:
# Display gradients
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dw: tensor(3.)
dy/db: tensor(1.)


## Problem Statement

We'll create a model that predicts crop yeilds for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

<img src="https://i.imgur.com/lBguUV9.png" width="500" />

In a **linear regression** model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.

<img src="https://i.imgur.com/mtkR2lB.png" width="540" >


**Our objective**: Find a suitable set of *weights* and *biases* using the training data, to make accurate predictions.

## Training Data
The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column per variable.

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [0]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

**Before we build a model, we need to convert inputs and targets to PyTorch tensors.**

In [63]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear Regression Model (from scratch)

The *weights* and *biases* can also be represented as matrices, initialized with random values. The first row of `w` and the first element of `b` are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [64]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.4804,  0.5091,  1.5005],
        [-0.0335,  2.4950,  0.6119]], requires_grad=True)
tensor([0.8160, 0.2564], requires_grad=True)


The *model* is simply a function that performs a matrix multiplication of the input `x` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

$$
\hspace{2.5cm} X \hspace{1.1cm} \times \hspace{1.2cm} W^T \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\times
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$

In [0]:
# Define the model
def model(x):
    return x @ w.t() + b

The matrix obtained by passing the input data to the model is a set of predictions for the target variables.

In [66]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[134.5216, 191.2902],
        [185.3721, 255.9330],
        [197.8669, 367.1661],
        [127.2319, 126.7671],
        [187.8788, 280.3014]], grad_fn=<ThAddBackward>)


In [67]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Because we've started with random weights and biases, the model does not a very good job of predicting the target varaibles.

## Loss Function

We can compare the predictions with the actual targets, using the following method: 
* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [0]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [69]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(16918.2871, grad_fn=<DivBackward0>)


The resulting number is called the **loss**, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model. 

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the `loss` w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [0]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors.

In [71]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.4804,  0.5091,  1.5005],
        [-0.0335,  2.4950,  0.6119]], requires_grad=True)
tensor([[ 7736.3286,  7537.4375,  4893.1230],
        [12722.3154, 14496.7441,  8665.0605]])


In [72]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.8160, 0.2564], requires_grad=True)
tensor([ 90.3743, 152.2915])


A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

* If a gradient element is **postive**, 
    * **increasing** the element's value slightly will **increase** the loss.
    * **decreasing** the element's value slightly will **decrease** the loss.

<img src="https://i.imgur.com/2H4INoV.png" width="400" />



* If a gradient element is **negative**,
    * **increasing** the element's value slightly will **decrease** the loss.
    * **decreasing** the element's value slightly will **increase** the loss.
    
<img src="https://i.imgur.com/h7E2uAv.png" width="400" />    

The increase or decrease is proportional to the value of the gradient.

Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [73]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [74]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[134.5216, 191.2902],
        [185.3721, 255.9330],
        [197.8669, 367.1661],
        [127.2319, 126.7671],
        [187.8788, 280.3014]], grad_fn=<ThAddBackward>)


In [75]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(16918.2871, grad_fn=<DivBackward0>)


In [0]:
# Compute gradients
loss.backward()

In [0]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [78]:
print(w)

tensor([[ 0.4031,  0.4338,  1.4516],
        [-0.1607,  2.3500,  0.5253]], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [79]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(11566.7275, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [0]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [81]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(180.4525, grad_fn=<DivBackward0>)


In [82]:
# Print predictions
preds

tensor([[ 60.2299,  67.3767],
        [ 88.9419,  91.9881],
        [ 98.4510, 157.4191],
        [ 38.5784,  19.0542],
        [103.5860, 114.5646]], grad_fn=<ThAddBackward>)

In [83]:
# Print targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

In [0]:
# Imports
import torch.nn as nn

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [0]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [0]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [88]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [89]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 73.,  67.,  43.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [ 22.,  37.],
         [ 56.,  70.],
         [119., 133.]])]

### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using `nn.Linear`.

In [90]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0370, -0.1672, -0.3510],
        [ 0.0935,  0.3218,  0.4441]], requires_grad=True)
Parameter containing:
tensor([0.3145, 0.3437], requires_grad=True)


### Optimizer
Instead of manually manipulating the weights & biases using gradients, we can use the optimizer `optim.SGD`.

In [0]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [0]:
# Import nn.functional
import torch.nn.functional as F

In [0]:
# Define loss function
loss_fn = F.mse_loss

In [94]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7971.5732, grad_fn=<MseLossBackward>)


### Train the model

We are ready to train the model now. We can define a utility function `fit` which trains the model for a given number of epochs.

In [0]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(inputs), targets))

In [96]:
# Train the model for 100 epochs
fit(100, model, loss_fn, opt)

Training loss:  tensor(24.9746, grad_fn=<MseLossBackward>)


In [97]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.6718,  71.2313],
        [ 79.5314,  99.3507],
        [121.5757, 133.9520],
        [ 29.7788,  42.2641],
        [ 92.3094, 113.8850],
        [ 58.6718,  71.2313],
        [ 79.5314,  99.3507],
        [121.5757, 133.9520],
        [ 29.7788,  42.2641],
        [ 92.3094, 113.8850],
        [ 58.6718,  71.2313],
        [ 79.5314,  99.3507],
        [121.5757, 133.9520],
        [ 29.7788,  42.2641],
        [ 92.3094, 113.8850]], grad_fn=<ThAddmmBackward>)

In [98]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Bonus: Feedfoward Neural Network

![ffnn](https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Multi-Layer_Neural_Network-Vector-Blank.svg/400px-Multi-Layer_Neural_Network-Vector-Blank.svg.png)

Conceptually, you think of feedforward neural networks as two or more linear regression models stacked on top of one another with a non-linear activation function applied between them.

<img src="https://cdn-images-1.medium.com/max/1600/1*XxxiA0jJvPrHEJHD4z893g.png" width="640">

To use a feedforward neural network instead of linear regression, we can extend the `nn.Module` class from PyTorch.

In [0]:
class SimpleNet(nn.Module):
    # Initialize the layers
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 3)
        self.act1 = nn.ReLU() # Activation function
        self.linear2 = nn.Linear(3, 2)
    
    # Perform the computation
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        return x

Now we can define the model, optimizer and loss function exactly as before.

In [0]:
model = SimpleNet()
opt = torch.optim.SGD(model.parameters(), 1e-5)
loss_fn = F.mse_loss

Finally, we can apply gradient descent to train the model using the same `fit` function defined earlier for linear regression.

<img src="https://i.imgur.com/g7Rl0r8.png" width="500">

In [101]:
fit(100, model, loss_fn, opt)

Training loss:  tensor(8.3765, grad_fn=<MseLossBackward>)
